In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report


In [3]:
# Loading data
df = pd.read_csv('Data\\balanced_augmented_Call_Conversation(Emotion).csv')
print(df.shape)

(602, 4)


In [4]:
df.head(3)

,VoiceClip_No,TextTranscript,label,processed_content
0,Voice52,the experience was your agent responded positi...,Non,the experience was your agent responded positi...
1,Voice38,get not working all,Non,get not working all
2,Voice199,can know the current price the fiber connection,Non,can know the current price the fiber connection


In [84]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

x = df['TextTranscript']
y = df['label']

# Text vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(x)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=24)

# SVM model with grid search
svm_classifier = SVC() #random_state=42)

# Hyperparameter grid to search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear'],
    'class_weight': [None, 'balanced']
}

grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

# Best hyperparameters from the grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the final model with the best hyperparameters on the full training dataset
final_svm_classifier = SVC(**best_params)
final_svm_classifier.fit(X_train, y_train)

# Predict sentiment on the testing dataset
y_pred_test = final_svm_classifier.predict(X_test)
y_pred_train = final_svm_classifier.predict(X_train)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

# Print accuracy with 4 decimal points
print(f"\nTraining Accuracy: {accuracy_train:.4f}")
print(f"Testing Accuracy: {accuracy_test:.4f}")

# Classification report for testing dataset
report_test = classification_report(y_test, y_pred_test)
print("\nClassification Report on Testing Data:")
print(report_test)


Best Hyperparameters: {'C': 1, 'class_weight': None, 'kernel': 'linear'}

Training Accuracy: 0.9958
Testing Accuracy: 0.9421

Classification Report on Testing Data:
              precision    recall  f1-score   support

         Agg       0.95      0.94      0.94        62
         Non       0.93      0.95      0.94        59

    accuracy                           0.94       121
   macro avg       0.94      0.94      0.94       121
weighted avg       0.94      0.94      0.94       121

